In [40]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures


In [3]:
train = pd.DataFrame(pd.read_csv('train.csv'))
test = pd.DataFrame(pd.read_csv('test.csv'))

In [8]:
train.columns = [i.lower().replace(' ', '') for i in train.columns]
test.columns = [i.lower().replace(' ', '') for i in test.columns]

In [9]:
train.head()

,id,pid,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,...,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [10]:
test.head()

,id,pid,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,...,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [64]:
#set X and y variables         
X_train_num = train.loc[:, [#'id',
                            '1stflrsf', '2ndflrsf', 'bsmtunfsf', 'bsmtfinsf1', 'garagearea',
                            'garageyrblt', 'grlivearea', 'lotfrontage', 'masvnrarea', 
                            'openporchsf', 'overallqual', 'screenporch', 'totalbsmtsf', 
                            'totrmsabvgrd', 'wooddecksf', 'yearbuilt'
                            #'saleprice'
]]

X_train_obj = pd.get_dummies(train.loc[:, [#'id',
            'kitchenqual', 'foundation', 'exterqual'
            ]], drop_first = False)

X_train_obj = X_train_obj.loc[:, ['kitchenqual_Ex', 'foundation_PConc', 'kitchenqual_TA', 'exterqual_TA', 
                               'exterqual_Ex', 'exterqual_Gd']]

X_train = pd.concat([X_train_num, X_train_obj], axis = 1)
y_train = train['saleprice']

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [65]:
X_train.fillna(value = 0, inplace = True)
X_train.isnull().sum()

#y_train.isnull().sum()

1stflrsf            0
2ndflrsf            0
bsmtunfsf           0
bsmtfinsf1          0
garagearea          0
garageyrblt         0
grlivearea          0
lotfrontage         0
masvnrarea          0
openporchsf         0
overallqual         0
screenporch         0
totalbsmtsf         0
totrmsabvgrd        0
wooddecksf          0
yearbuilt           0
kitchenqual_Ex      0
foundation_PConc    0
kitchenqual_TA      0
exterqual_TA        0
exterqual_Ex        0
exterqual_Gd        0
dtype: int64

In [66]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, test_size = 0.25)

print(X_train_train.shape, y_train_train.shape)
print(X_train_test.shape, y_train_test.shape)

(1538, 22) (1538,)
(513, 22) (513,)


In [67]:
#calculate baseline accuracy: mean saleprice in y_train
y_train_train.mean().round(2)

179117.93

In [68]:
X_train_train.head()

,1stflrsf,2ndflrsf,bsmtunfsf,bsmtfinsf1,garagearea,garageyrblt,grlivearea,lotfrontage,masvnrarea,openporchsf,...,totalbsmtsf,totrmsabvgrd,wooddecksf,yearbuilt,kitchenqual_Ex,foundation_PConc,kitchenqual_TA,exterqual_TA,exterqual_Ex,exterqual_Gd
284,812,670,384.0,0.0,392.0,1992.0,0.0,0.0,170.0,25,...,384.0,7,100,1992,0,1,0,1,0,0
1062,816,0,242.0,574.0,240.0,1949.0,0.0,60.0,0.0,0,...,816.0,4,228,1934,0,0,1,1,0,0
1539,992,0,234.0,758.0,294.0,1967.0,0.0,68.0,0.0,0,...,992.0,6,204,1967,0,0,0,1,0,0
1743,804,0,278.0,510.0,240.0,1940.0,0.0,60.0,294.0,0,...,788.0,4,0,1940,0,0,0,1,0,0
1748,956,1128,300.0,656.0,618.0,2004.0,0.0,76.0,0.0,45,...,956.0,8,0,2004,0,1,0,0,0,1


In [69]:
ss = StandardScaler()
pf = PolynomialFeatures()
#lr = LinearRegression()
lm = Lasso()

In [70]:
Lasso()

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [71]:
pipe = Pipeline([
    #set tuples: one for each instance to be incorporated into pipeline
    ('ss', ss),
    ('pf', pf),
    ('lm', lm)
])

In [73]:
pipe.fit(X_train_train, y_train_train)
pipe.score(X_train_train, y_train_train)

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.9300806909440537

In [74]:
pipe.score(X_train_test, y_train_test)

0.8496645105275729

In [84]:
params = {
    'lm__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 
                  0.6, 0.7, 0.8, 0.9, 1.0]
}

pipe_out = GridSearchCV(pipe, param_grid = params)

In [125]:
pipe_out.fit(X_train_train, y_train_train)
print(pipe_out.best_score_)
print(pipe_out.best_params_)

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: 

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.7143560080087635
{'lm__alpha': 1.0}


/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [87]:
pipe_out.score(X_train_test, y_train_test)

0.8496645105275729

In [88]:
X_test_num = test.loc[:, [#'id',
                            '1stflrsf', '2ndflrsf', 'bsmtunfsf', 'bsmtfinsf1', 'garagearea',
                            'garageyrblt', 'grlivearea', 'lotfrontage', 'masvnrarea', 
                            'openporchsf', 'overallqual', 'screenporch', 'totalbsmtsf', 
                            'totrmsabvgrd', 'wooddecksf', 'yearbuilt'
                            #'saleprice'
]]

X_test_obj = pd.get_dummies(test.loc[:, [#'id',
            'kitchenqual', 'foundation', 'exterqual'
            ]], drop_first = False)

X_test_obj = X_test_obj.loc[:, ['kitchenqual_Ex', 'foundation_PConc', 'kitchenqual_TA', 'exterqual_TA', 
                               'exterqual_Ex', 'exterqual_Gd']]

X_test = pd.concat([X_test_num, X_test_obj], axis = 1)

/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Users/chelen/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [92]:
X_test.fillna(value = 0, inplace = True)
X_test.isnull().sum()

1stflrsf            0
2ndflrsf            0
bsmtunfsf           0
bsmtfinsf1          0
garagearea          0
garageyrblt         0
grlivearea          0
lotfrontage         0
masvnrarea          0
openporchsf         0
overallqual         0
screenporch         0
totalbsmtsf         0
totrmsabvgrd        0
wooddecksf          0
yearbuilt           0
kitchenqual_Ex      0
foundation_PConc    0
kitchenqual_TA      0
exterqual_TA        0
exterqual_Ex        0
exterqual_Gd        0
dtype: int64

In [94]:
predictions = pipe_out.predict(X_test)

In [105]:
X_test['Id'] = test['id']
X_test['Saleprice'] = predictions.round(2)

In [106]:
sample8 = X_test.loc[:, ['Id', 'Saleprice']]

In [107]:
sample8.set_index('Id', inplace = True)
sample8.sort_index(axis = 0, inplace = True)

In [108]:
sample8.to_csv('sample8_hc')
pd.read_csv('sample8_hc').head()